In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3 
import os
import matplotlib.pyplot as plt
from PIL import Image
import glob
from html import escape
%matplotlib inline


def news_crawl(query, start_date, end_date, sort_type, max_page):
    dbpath = "news_info.db" 
    conn = sqlite3.connect(dbpath)
    cur = conn.cursor() 

    script = """
    DROP TABLE IF EXISTS news_crawl;

    CREATE TABLE news_crawl(
      id INTEGER PRIMARY KEY AUTOINCREMENT,  -- 뉴스의 ID 값
      date TEXT,                             -- 뉴스의 작성일
      title TEXT,                            -- 뉴스의 제목
      summary TEXT,                          -- 뉴스의 요약
      link TEXT,                             -- 뉴스의 원문 링크
      detailed_link TEXT,                    -- 뉴스의 상세 페이지 링크
      content TEXT,                          -- 뉴스의 본문 내용
      press TEXT                             -- 뉴스의 언론사
    );
    """
    cur.executescript(script)
        
    if query == '':
        query = '데이터 분석'
    if len(start_date) != 10:
        start_date = '2024.01.01'
    if len(end_date) != 10:
        end_date = '2024.01.11'
    if sort_type not in ['0', '1', '2']:
        sort_type = '0'
    start_date = start_date.replace(".", "")
    end_date = end_date.replace(".", "")
    start_page=1

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

    for page in range(1, max_page + 1):
        current_call = 1 + (page - 1) * 10
        url = "https://search.naver.com/search.naver?where=news&query=" + query \
              + "&sort=" + sort_type \
              + "&ds=" + start_date \
              + "&de=" + end_date \
              + "&start=" + str(current_call)

        web = requests.get(url, headers=headers).content
        source = BeautifulSoup(web, 'html.parser')

        # 각 페이지 내의 모든 뉴스 기사를 순회합니다.
        for article in source.find_all('div', {'class': 'news_area'}):
            title = article.find('a', {'class': 'news_tit'}).get('title').replace("'", "''")
            link = article.find('a', {'class': 'news_tit'}).get('href').replace("'", "''")
            summary = article.find('a', {'class': 'api_txt_lines dsc_txt_wrap'}).get_text().replace("'", "''")
            detailed_url = ""
            #네이버 뉴스에 등록된 뉴스만 사용
            for urls in article.find_all('a', {'class': 'info'}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    detailed_url = urls["href"].replace("'", "''")
                    break
            if detailed_url:
                response = requests.get(detailed_url, headers=headers)
                soup = BeautifulSoup(response.content, 'html.parser')
                press_company = soup.find('em', {'class':'media_end_linked_more_point'}).get_text()
                date = soup.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()
                content_area = soup.find(id='dic_area')
                news_content = content_area.get_text(strip=True).replace("'", "''") if content_area else "본문을 찾을 수 없습니다."
                news_content = escape(news_content)

                base_sql = "INSERT INTO news_crawl(date, title, summary, link, detailed_link, content, press) values('{}','{}', '{}', '{}', '{}', '{}','{}')"
                sql_query = base_sql.format(date, title, summary, link, detailed_url, news_content, press_company)
                cur.execute(sql_query)
                conn.commit()
            

    conn.close()

# 사용자 입력을 통한 크롤링 실행
query = input('크롤링하고 싶은 뉴스 검색어를 입력해주세요: ')
start_date = input('크롤링하고 싶은 뉴스의 시작 날짜를 입력해주세요(ex:2024.01.01): ')
end_date = input('크롤링하고 싶은 뉴스의 종료 날짜를 입력해주세요(ex:2024.01.01): ')
sort_type = int(input('크롤링하고 싶은 뉴스의 정렬방법을 입력해주세요(관련도순 = 0  최신순 = 1  오래된순 = 2): '))
max_page = int(input('크롤링하고 싶은 뉴스의 페이지 수를 입력해주세요: '))
news_crawl(query, start_date, end_date, sort_type, max_page)


크롤링하고 싶은 뉴스 검색어를 입력해주세요: 데이터분석
크롤링하고 싶은 뉴스의 시작 날짜를 입력해주세요(ex:2024.01.01): 2023.12.17
크롤링하고 싶은 뉴스의 종료 날짜를 입력해주세요(ex:2024.01.01): 2024.01.15
크롤링하고 싶은 뉴스의 정렬방법을 입력해주세요(관련도순 = 0  최신순 = 1  오래된순 = 2): 0
크롤링하고 싶은 뉴스의 페이지 수를 입력해주세요: 10


In [20]:
#데이터 저장
import pandas as pd
import sqlite3

# 데이터베이스에서 데이터 읽기
conn = sqlite3.connect('news_info.db')
query = "SELECT * FROM news_crawl"
df = pd.read_sql_query(query, conn)
conn.close()

# 데이터를 CSV 파일로 변환
csv_file_path = "news_data.csv"
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

In [24]:
#html + gpt api
from flask import Flask, request, jsonify, render_template
import sqlite3
import html
from openai import OpenAI
import os

app = Flask(__name__)

# API 키를 파일에서 읽기
with open('OPENAI_API_KEY.txt', 'r') as file:
    api_key = file.read().strip()
    
client = OpenAI(api_key=api_key)
def generate_gpt_content(news_content):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"'{news_content}' 위 내용을 바탕으로 한국어로 개선된 뉴스본문을 작성해. 그리고 50자 이내의 요약과 30자 이내의 제목도 생성해. 반드시 뉴스본문 앞에는 gpt_content: 를 요약 앞에는 gpt_summary: 를, 제목 앞에는 gpt_title: 을 붙여. 그리고 제목, 요약, 본문 순으로 나에게 보여줘."
            },
        ]
    )
    return response.choices[0].message.content

@app.route('/')
def show_news():
    # 데이터베이스에서 뉴스 데이터를 가져옵니다.
    conn = sqlite3.connect('news_info.db')
    cur = conn.cursor()
    cur.execute("SELECT * FROM news_crawl")
    news_data = cur.fetchall()
    conn.close()

    decoded_news_data = []
    for news in news_data:
        decoded_content = news[6]
        decoded_news = list(news)
        decoded_news[6] = decoded_content
        decoded_news_data.append(decoded_news)
        # HTML 템플릿에 데이터를 전달합니다.
    return render_template('news_template.html', news_data=decoded_news_data)
@app.route('/process', methods=['POST'])

def process():
    news_content = request.form['content']
    decoded_content = html.unescape(news_content)
    app.logger.info('Received content: %s', decoded_content)

    gpt_response = generate_gpt_content(decoded_content)

    return jsonify({'gpt_result': gpt_response})

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Jan/2024 16:16:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 16:16:18] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2024 16:17:01] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 16:17:40] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2024 16:17:48] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 16:18:37] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2024 16:19:56] "POST /process HTTP/1.1" 200 -
